In [1]:
import serial
import time
from time import sleep
import threading
import sys
import struct
import binascii
import math
from time import gmtime, strftime

In [8]:
# the configuration of the serial port 
COM_PORT = 'COM19' 
#BAUD_RATES = 115200
BAUD_RATES = 256000

sampleSec = 5
loopNum = 0

readbyte  = 11000 # Unit in byte # 1000 Hz * 3 axis * 4 byte(float)


while loopNum < sampleSec:
    offsetNum = 9999
    while (offsetNum == 9999):
        try:
            ser = serial.Serial(COM_PORT, BAUD_RATES, timeout=1, stopbits=1, bytesize=8)
            data_bin = ser.read(readbyte + 3) # Unit in byte # Leave 3 bytes to check the offset
            #with open('./raw.bin', 'wb') as fid:
            #    fid.write(data)    
            ser.close()
            #print('Done.')

            '''
            fileName = 'raw.bin'
            with open(fileName, 'rb') as fid:
                data_bin = fid.read()
                print("data size: {} bytes".format(sys.getsizeof(data_bin)))
                print("----------------")
            '''
            
            for ii in range(0,3):
                floatbyte = readbyte // 4
                fmt = str(floatbyte) + "f"
                data_decode = struct.unpack_from(fmt, data_bin, offset = ii) # offset unit: byte # float 4 bytes
                print("the offset of the pointer within the file: {} byte".format(ii))
                logVal = abs(math.log(abs(data_decode[0]), 10)) +\
                         abs(math.log(abs(data_decode[1]), 10)) +\
                         abs(math.log(abs(data_decode[2]), 10))        

                if logVal < 5:
                    offsetNum = ii
                    print(data_decode[0:3])
                    print("log value: {}".format(logVal))
                    print("offset: {}".format(offsetNum))
                    print("----------------")
                    break

                print(data_decode[0:3])
                print("log value: {}".format(logVal))
                print("offset: {}".format(offsetNum))
                print("----------------")
        except:
            print('Failed to open the port.')

    data_decode = struct.unpack_from(fmt, data_bin, offset = offsetNum)
    data_decode = list(data_decode)

    if data_decode[0] > 900:
        data_decode = data_decode[1:]
    elif data_decode[1] > 900:
        data_decode = data_decode[2:]

    if (len(data_decode) % 3) > 0:
        lastGroup = len(data_decode) - len(data_decode) % 3
        data_decode = data_decode[:lastGroup]

    now = strftime("%Y%m%d%H%M%S")
    queryFid = "acc3_{}_{}.txt".format(now, loopNum)
    with open(queryFid, "w") as file:
        file.write(str(data_decode))
    
    print("Done. loop = {}".format(loopNum))
    loopNum += 1
    #time.sleep(0.1)



the offset of the pointer within the file: 0 byte
(3.081199884414673, 984.003173828125, 8.475196838378906)
log value: 4.409866165695794
offset: 0
----------------
Done. loop = 0
the offset of the pointer within the file: 0 byte
(2.3799551997766346e+31, -3.5026210695930767e-37, 4.106972420841096e-20)
log value: 87.2186538397051
offset: 9999
----------------
the offset of the pointer within the file: 1 byte
(974.3468017578125, 4.0915985107421875, 2.4883999824523926)
log value: 3.9965267654150276
offset: 1
----------------
Done. loop = 1
the offset of the pointer within the file: 0 byte
(7.008796691894531, 0.41359996795654297, 974.8148193359375)
log value: 4.217985088700116
offset: 0
----------------
Done. loop = 2
the offset of the pointer within the file: 0 byte
(-0.43799498677253723, 1.9739654247112604e-13, -292631969792.0)
log value: 24.529513089570948
offset: 9999
----------------
the offset of the pointer within the file: 1 byte
(-8.680034468966369e-11, 1.349795538067478e+31, -26948